In [49]:
import sqlite3
import pandas as pd
import numpy as np
import cvxpy

In [112]:
proj = pd.concat([pd.read_csv("proj.csv"), pd.read_csv("proj2.csv")])
proj = proj[proj["FantasyPointsFanDuel"] > 0]

slate = pd.read_csv("slate.csv")
slate["Name"] = names = slate["First Name"] + " " + slate["Last Name"]
# Merge expected number of fantasy points to current slate. Inner-join,
# so any players on the slate not in 'proj' get dropped
slate = slate.merge(proj[["Name", "FantasyPointsFanDuel"]], on="Name")

In [217]:
# For each player(row), denote by a 1 positions that player can play
valid_positions = slate["Position"].str.get_dummies(sep="/")
# Combine 1B and C to one column where a 1 is present if the player can play
# either or both
valid_positions["C/1B"] = (test["1B"].astype(bool) | test["C"].astype(bool)).astype(int)
valid_positions = valid_positions.drop(columns=["1B", "C"])
# Any non-pitcher player can fill the UTIL position
valid_positions["UTIL"] = valid_positions.drop(columns="P").astype(bool).any(axis=1).astype(int)
# Reordering columns
valid_positions = valid_positions[["P", "C/1B", "2B", "3B", "SS", "OF", "UTIL"]]

In [201]:
# Each player is a row, each column represents a position
# Order: P, C/1B, 2B, 3B, SS, OF, UTIL
selection = cvxpy.Variable((len(slate), 7), boolean=True)

# Each player can only be selected for 1 position
individual = cvxpy.sum(selection, axis=1) == 1
# Ensure that each player is only selected for positions they can play
feasible_positions = selection <= valid_positions
# Ensure that we select the proper number of players for each position
position_limits = [1, 1, 1, 1, 1, 3, 1]
positions = cvxpy.sum(selection, axis=0) == position_limits

player_teams = pd.get_dummies(slate["Team"])
team_indicator = cvxpy.Variable(len(slate["Team"].unique()), boolean=True)
teams = cvxpy.sum(selection, axis=1) @ player_teams <= 

In [236]:
slate["Team"].unique()

array(['STL', 'LAA', 'MIL', 'KC', 'LAD', 'HOU', 'OAK', 'NYY', 'CWS',
       'BOS', 'ATL', 'BAL', 'TOR', 'SF', 'ARI', 'SEA', 'MIN', 'PIT',
       'NYM', 'TEX', 'COL', 'CHC'], dtype=object)

In [227]:
test = np.random.randint(0, 2, 202)

In [234]:
test @ pd.get_dummies(slate["Team"])

ARI    5
ATL    4
BAL    4
BOS    5
CHC    4
COL    6
CWS    2
HOU    1
KC     6
LAA    7
LAD    2
MIL    4
MIN    4
NYM    5
NYY    7
OAK    6
PIT    4
SEA    3
SF     6
STL    6
TEX    4
TOR    4
dtype: int64

https://math.stackexchange.com/questions/2220355/involving-indicator-function-as-a-constraint-in-a-lp-problem